In [1]:
try:
  # We must install required packages if we are in Google Colab
  import google.colab
  %pip install roboticstoolbox-python>=1.0.2
except:
  # We are not in Google Colab
  # Apply custon style to notebook
  from IPython.core.display import HTML
  import pathlib
  styles_path = pathlib.Path(pathlib.Path().absolute(), "style", "style.css")
  styles = open(styles_path, "r").read()
  HTML(f"<style>{styles}</style>")

# 2.0 Higher Order Derivatives

$\large{\text{Manipulator Differential Kinematics}} \\ \large{\text{Part II: Acceleration and Advanced Applications}}$

$\text{By Jesse Haviland and Peter Corke}$

<br>

The sections of the Tutorial paper related to this notebook are listed next to each contents entry.
It is beneficial to read these sections of the paper before attempting the notebook Section.

### Contents

[2.1 Higher Order Derivatives](#pre)
* Part I of the Tutorial
  * Deriving the Manipulator Jacobian
  * First Derivative of an Elementary Transform
  * The Manipulator Jacobian
  * Fast Manipulator Jacobian


In [2]:
# We will do the imports required for this notebook here

# numpy provides import array and linear algebra utilities
import numpy as np

# the robotics toolbox provides robotics specific functionality
import roboticstoolbox as rtb

<br>

<a id='pre'></a>
### 2.1 Higher Order Derivatives
---


Starting from $n = 3$, obtaining the matrix of $n^{th}$ partial derivative of the manipulator kinematics can be obtained using the product rule on the $(n - 1)^{th}$ partial derivative while considering their partitioned form.

For example, to obtain the $3^{rd}$ partial derivative, we take the partial derivative of the manipulator Hessian with respect to the joint coordinates, in its partitioned form

\begin{align*}
    \frac{\partial \bf{H}_{jk}(\bf{q})}
         {\partial q_l}
    &=
    \begin{pmatrix}
        \dfrac{\partial \bf{H}_{a_{jk}}(\bf{q})}
              {\partial q_l} \\
        \dfrac{\partial \bf{H}_{\alpha_{jk}}(\bf{q})}
              {\partial q_l}
    \end{pmatrix}  \\
    &=
    \begin{pmatrix}
        \dfrac{\partial }
              {\partial q_l}
        \left( \bf{J}_{\omega_k} \times \bf{J}_{\omega_j} \right) \\
        \dfrac{\partial}
              {\partial q_l}
        \left( \bf{J}_{\omega_k} \times \bf{J}_{v_j} \right)
    \end{pmatrix}  \\
    &=
    \begin{pmatrix}
        \left( \bf{H}_{\omega_{kl}} \times \bf{J}_{\omega_j} \right) +
        \left( \bf{J}_{\omega_k} \times \bf{H}_{\omega_{jl}} \right) \\
        \left( \bf{H}_{\omega_{kl}} \times \bf{J}_{v_j} \right) +
        \left( \bf{J}_{\omega_k} \times \bf{H}_{v_{jl}} \right)
    \end{pmatrix}
\end{align*}

where $\frac{\partial \bf{H}_{jk}(\bf{q})}{\partial q_l} \in \mathbb{R}^{6}$. Continuing, we obtain the following

\begin{align*}
    \frac{\partial \bf{H}_{jk}(\bf{q})}
         {\partial \bf{q}}
    &=
    \begin{pmatrix}
        \dfrac{\partial \bf{H}_{jk}(\bf{q})}
            {\partial q_0} & \cdots & 
        \dfrac{\partial \bf{H}_{jk}(\bf{q})}
            {\partial q_n}
    \end{pmatrix}
\end{align*}

where $\frac{\partial \bf{H}_{jk}(\bf{q})}{\partial q_l} \in \mathbb{R}^{6 \times n}$,

\begin{align*}
    \frac{\partial \bf{H}_{j}(\bf{q})}
        {\partial \bf{q}}
    &=
    \begin{pmatrix}
        \dfrac{\partial \bf{H}_{j_0}(\bf{q})}
            {\partial \bf{q}} & \cdots & 
        \dfrac{\partial \bf{H}_{j_n}(\bf{q})}
            {\partial \bf{q}}
    \end{pmatrix}
\end{align*}

where $\frac{\partial \bf{H}_{jk}(\bf{q})}{\partial \bf{q}} \in \mathbb{R}^{6 \times n \times n}$, and finally 

\begin{align*}
    \frac{\partial \bf{H}(\bf{q})}
         {\partial \bf{q}}
    &=
    \begin{pmatrix}
        \dfrac{\partial \bf{H}_{0}(\bf{q})}
            {\partial \bf{q}} & \cdots & 
        \dfrac{\partial \bf{H}_{n}(\bf{q})}
            {\partial \bf{q}}
    \end{pmatrix}
\end{align*}

where $\frac{\partial \bf{H}(\bf{q})}{\partial \bf{q}} \in \mathbb{R}^{6 \times n \times n \times n}$ is the 4-dimensional tensor representing the $3^{rd}$ partial derivative of the manipulator kinematics.

Note that the function has $\mathcal{O}(n^{order})$ time complexity, where $order$ represents the order of the partial derivative being calculated.

In [3]:
def partial_fkine0(ets, q, n):
    r"""
    Manipulator Forward Kinematics nth Partial Derivative

    :return: The nth Partial Derivative of the forward kinematics
    """

    # Calculate the Jacobian and Hessian
    J = ets.jacob0(q)
    H = ets.hessian0(q)

    # A list of derivatives, starting with the jacobian and hessian
    dT = [J, H]

    # The tensor dimensions of the latest derivative
    # Set to the current size of the Hessian
    size = [ets.n, 6, ets.n]

    # An array which keeps track of the index of the partial derivative
    # we are calculating
    # It stores the indices in the order: "j, k, l. m, n, o, ..."
    # where count is extended to match oder of the partial derivative
    count = np.array([0, 0])

    # The order of derivative for which we are calculating
    # The Hessian is the 2nd-order so we start with c = 2
    c = 2

    def add_indices(indices, c):
        total = len(indices * 2)
        new_indices = []

        for i in range(total):
            j = i // 2
            new_indices.append([])
            new_indices[i].append(indices[j][0].copy())
            new_indices[i].append(indices[j][1].copy())

            if i % 2 == 0:
                # if even number
                new_indices[i][0].append(c)
            else:
                # if odd number
                new_indices[i][1].append(c)

        return new_indices

    def add_pdi(pdi):
        total = len(pdi * 2)
        new_pdi = []

        for i in range(total):
            j = i // 2
            new_pdi.append([])
            new_pdi[i].append(pdi[j][0])
            new_pdi[i].append(pdi[j][1])

            # if even number
            if i % 2 == 0:
                new_pdi[i][0] += 1
            # if odd number
            else:
                new_pdi[i][1] += 1

        return new_pdi

    # these are the indices used for the hessian
    indices = [[[1], [0]]]

    # The partial derivative indices (pdi)
    # the are the pd indices used in the cross products
    pdi = [[0, 0]]

    # The length of dT correspods to the number of derivatives we have calculated
    while len(dT) != n:

        # Add to the start of the tensor size list 
        size.insert(0, ets.n)

        # Add an axis to the count array
        count = np.concatenate(([0], count))

        # This variables corresponds to indices within the previous partial derivatives
        # to be cross prodded
        # The order is: "[j, k, l, m, n, o, ...]"
        # Although, our partial derivatives have the order: pd[..., o, n, m, l, k, cartesian DoF, j]
        # For example, consider the Hessian Tensor H[n, 6, n], the index H[k, :, j]. This corrsponds
        # to the second partial derivative of the kinematics of joint j with respect to joint k.
        indices = add_indices(indices, c)

        # This variable corresponds to the indices in Td which corresponds to the 
        # partial derivatives we need to use
        pdi = add_pdi(pdi)

        c += 1

        # Allocate our new partial derivative tensor
        pd = np.zeros(size)

        # We need to loop n^c times
        # There are n^c columns to calculate
        for _ in range(ets.n**c):
            
            # Allocate the rotation and translation components
            rot = np.zeros(3)
            trn = np.zeros(3)

            # This loop calculates a single column ([trn, rot]) of the tensor for dT(x)
            for j in range(len(indices)):
                pdr0 = dT[pdi[j][0]]
                pdr1 = dT[pdi[j][1]]

                idx0 = count[indices[j][0]]
                idx1 = count[indices[j][1]]

                # This is a list of indices selecting the slices of the previous tensor
                idx0_slices = np.flip(idx0[1:])
                idx1_slices = np.flip(idx1[1:])

                # This index selecting the column within the 2d slice of the previous tensor
                idx0_n = idx0[0]
                idx1_n = idx1[0]

                # Use our indices to select the rotational column from pdr0 and pdr1
                col0_rot = pdr0[(*idx0_slices, slice(3, 6), idx0_n)]
                col1_rot = pdr1[(*idx1_slices, slice(3, 6), idx1_n)]

                # Use our indices to select the translational column from pdr1
                col1_trn = pdr1[(*idx1_slices, slice(0, 3), idx1_n)]

                # Perform the cross product as described in the maths above
                rot += np.cross(col0_rot, col1_rot)
                trn += np.cross(col0_rot, col1_trn)

            pd[(*np.flip(count[1:]), slice(0, 3), count[0])] = trn
            pd[(*np.flip(count[1:]), slice(3, 6), count[0])] = rot

            count[0] += 1
            for j in range(len(count)):
                if count[j] == ets.n:
                    count[j] = 0
                    if j != len(count) - 1:
                        count[j + 1] += 1

        dT.append(pd)

    return dT[-1]

Lets try it out

In [4]:
# Make a panda robot
panda = rtb.models.Panda()

# Get the ets of our robot
ets = panda.ets()

# Make a joint coordinate vector for the Panda
q = [0.21, -0.03, 0.35, -1.90, -0.04, 1.96, 1.36]

In [5]:
# Lets do the third-order differential dinematics
pd3 = partial_fkine0(ets, q, 3)

# When using the Panda, this will have dimensions 7 x 7 x 6 x 7
print(pd3.shape)

(7, 7, 6, 7)


In [6]:
# We can view the tensor, but it's very large! It has 49 6x7 slices
print(np.round(pd3, 2))

[[[[ 0.3  -0.08  0.3  ...  0.09 -0.17  0.  ]
   [-0.49 -0.02 -0.49 ... -0.14 -0.11  0.  ]
   [-0.    0.   -0.   ... -0.    0.    0.  ]
   [ 0.    0.21  0.03 ... -0.81 -0.54 -0.1 ]
   [ 0.   -0.98  0.01 ... -0.5   0.84 -0.01]
   [ 0.   -0.   -0.   ...  0.    0.    0.  ]]

  [[ 0.    0.11  0.   ...  0.   -0.02  0.  ]
   [-0.   -0.53 -0.01 ... -0.01  0.1   0.  ]
   [ 0.    0.    0.   ...  0.    0.    0.  ]
   [-0.21  0.   -0.21 ...  0.06 -0.01  0.21]
   [ 0.98  0.    0.98 ... -0.29  0.05 -0.97]
   [ 0.    0.    0.   ...  0.    0.   -0.  ]]

  [[ 0.3  -0.06  0.3  ...  0.09 -0.17  0.  ]
   [-0.49 -0.01 -0.49 ... -0.14 -0.11  0.  ]
   [-0.    0.   -0.   ... -0.    0.    0.  ]
   [-0.03  0.21  0.   ... -0.8  -0.54 -0.07]
   [-0.01 -0.98  0.   ... -0.5   0.84 -0.  ]
   [ 0.   -0.    0.   ...  0.    0.    0.  ]]

  ...

  [[-0.09 -0.41 -0.09 ... -0.03  0.14  0.  ]
   [ 0.14 -0.27  0.14 ...  0.04  0.09  0.  ]
   [ 0.    0.    0.   ...  0.    0.    0.  ]
   [ 0.81 -0.06  0.8  ...  0.    0.2  -0.7

In [7]:
# We can work out any order of the differential kinematics, but the
# higher we go, the loger it will take
# Lets do the fifth-order differential dinematics
pd5 = partial_fkine0(ets, q, 5)

# View the dimension
print(pd5.shape)

(7, 7, 7, 7, 6, 7)
